In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from google.colab import drive
from numpy.random import seed

In [2]:
drive.mount('/gdrive')
%cd /gdrive/MyDrive/F21DL_CW2/Data/CSV

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/F21DL_CW2/Data/CSV


In [3]:
X_train = pd.read_csv('x_train_gr_smpl.csv')
X_test = pd.read_csv('x_test_gr_smpl.csv')

X = pd.concat([X_train, X_test]).reset_index().drop(columns = ['index'])

y_train = pd.read_csv('y_train_smpl.csv')
y_test = pd.read_csv('y_test_smpl.csv')

y = pd.concat([y_train, y_test]).reset_index().drop(columns = ['index'])

X_train_4000, X_test_4000, y_train_4000, y_test_4000 = train_test_split(X, y, test_size=0.55, random_state=42, stratify = y)
X_train_9000, X_test_9000, y_train_9000, y_test_9000 = train_test_split(X, y, test_size=0.95, random_state=42, stratify = y)

X_train_4000 = X_train_4000.reset_index().drop(columns = ['index'])
X_test_4000 = X_test_4000.reset_index().drop(columns = ['index'])
y_train_4000 = y_train_4000.reset_index().drop(columns = ['index'])
y_test_4000 = y_test_4000.reset_index().drop(columns = ['index'])

X_train_9000 = X_train_9000.reset_index().drop(columns = ['index'])
X_test_9000 = X_test_9000.reset_index().drop(columns = ['index'])
y_train_9000 = y_train_9000.reset_index().drop(columns = ['index'])
y_test_9000 = y_test_9000.reset_index().drop(columns = ['index'])

X_train_tensor = tf.reshape(np.array(X_train), shape = (9690, 48, 48, 1))
X_test_tensor = tf.reshape(np.array(X_test), shape = (3090, 48, 48, 1))
X_train_4000_tensor = tf.reshape(np.array(X_train_4000), shape = (5750, 48, 48, 1))
X_test_4000_tensor = tf.reshape(np.array(X_test_4000), shape = (7030, 48, 48, 1))
X_train_9000_tensor = tf.reshape(np.array(X_train_9000), shape = (639, 48, 48, 1))
X_test_9000_tensor = tf.reshape(np.array(X_test_9000), shape = (12141, 48, 48, 1))


In [4]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state = 42)
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, min_delta = 0.0001, mode = 'min', restore_best_weights = True)
seed(42)

In [5]:
beta1 = [0.9, 0.09, 9]
beta2 = [0.999, 0.0999, 9.99]
lr = [0.001, 0.01, 0.1]

conv_kernels = [(4, 4), (3, 3)]
mp_kernels = [(3, 3), (2, 2)]

num_feature_maps = [64, 128, 256]

flattened_layers = [[512, 128], [1024, 512, 128], [2048, 1024, 512, 128]]

In [6]:
params_4000 = list()
acc_4000 = list()
class_reports_4000 = list()
%cd Data_from_cnn_tuning/testing_4000/

/gdrive/MyDrive/F21DL_CW2/Data/CSV/Data_from_cnn_tuning/testing_4000


In [7]:
for b1 in beta1:
  for b2 in beta2:
    for r in lr:
      adm = tf.keras.optimizers.Adam(learning_rate=r, beta_1=b1, beta_2=b2)
      for ck in range(len(conv_kernels)):
        for mk in range(len(mp_kernels)):
          for nf1 in num_feature_maps:
            for nf2 in num_feature_maps:
              model = tf.keras.Sequential([tf.keras.layers.Conv2D(nf1, conv_kernels[ck], activation='relu', input_shape=(48, 48, 1))])
              model.add(tf.keras.layers.MaxPooling2D(mp_kernels[mk]))
              model.add(tf.keras.layers.Conv2D(nf2, conv_kernels[ck], activation='relu'))
              model.add(tf.keras.layers.MaxPooling2D(mp_kernels[mk]))
              model.add(tf.keras.layers.Flatten())
              for l in range(len(flattened_layers)):
                for n in flattened_layers[l]:
                  model.add(tf.keras.layers.Dense(n, activation='relu'))
                model.add(tf.keras.layers.Dense(10, activation='softmax'))

                model.compile(optimizer=adm,
                              loss='sparse_categorical_crossentropy',
                              metrics=['accuracy'])
                
                params_4000.append([b1, b2, r, ck, mk, nf1, nf2, l])
                print(params_4000[-1])

                model.fit(x=X_train_4000_tensor,y=y_train_4000,epochs=5, callbacks = [es], verbose = 0)

                _, accu = model.evaluate(X_test_4000_tensor, y_test_4000, verbose = 0)

                acc_4000.append(accu)

                y_pred = model.predict(X_test_4000_tensor, verbose = 0)
                y_pred_bool = np.argmax(y_pred, axis=1)
                cr = pd.DataFrame(classification_report(y_test_4000, y_pred_bool, output_dict=True)).T
                auc_p_c = list()

                for i in range(10):
                  fpr, tpr, _ = roc_curve(np.array(y_test_4000), y_pred_bool, pos_label=i)
                  auc_p_c.append(auc(fpr, tpr))
                  
                cr['auc'] = auc_p_c + [0, 0, 0]
                cr.loc['weighted avg', 'auc'] = (cr.loc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'auc'] * cr.loc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'support']).sum() / cr.loc['macro avg', 'support']
                cr.loc['macro avg', 'auc'] = (cr.loc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'auc']).mean()
                class_reports_4000.append(cr)

                print(accu)
                
                cr.to_csv('testing_4000_b1_' + str(b1) + '_b2_' + str(b2) + '_lr_' + str(r) + '_ck_' + str(ck) + '_mk_' + str(mk) + '_nf1_' + str(nf1) + '_nf2_' + str(nf2) + '_l_' + str(l) + '.csv')

[0.9, 0.999, 0.001, 0, 0, 64, 64, 0]
0.9621621370315552
[0.9, 0.999, 0.001, 0, 0, 64, 64, 1]
0.933570384979248
[0.9, 0.999, 0.001, 0, 0, 64, 64, 2]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.8847795128822327
[0.9, 0.999, 0.001, 0, 0, 64, 128, 0]
0.8180654048919678
[0.9, 0.999, 0.001, 0, 0, 64, 128, 1]
0.8513513803482056
[0.9, 0.999, 0.001, 0, 0, 64, 128, 2]
0.8763869404792786
[0.9, 0.999, 0.001, 0, 0, 64, 256, 0]
0.9120910167694092
[0.9, 0.999, 0.001, 0, 0, 64, 256, 1]
0.9126600027084351
[0.9, 0.999, 0.001, 0, 0, 64, 256, 2]
0.9321479201316833
[0.9, 0.999, 0.001, 0, 0, 128, 64, 0]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.23470839858055115
[0.9, 0.999, 0.001, 0, 0, 128, 64, 1]


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.23001421988010406
[0.9, 0.0999, 0.001, 0, 0, 128, 64, 2]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.23001421988010406
[0.9, 0.0999, 0.001, 0, 0, 128, 128, 0]


TypeError: ignored

In [20]:
X_train_9000_tensor[0]

<tf.Tensor: shape=(48, 48, 1), dtype=float64, numpy=
array([[[ 58.],
        [ 61.],
        [ 65.],
        ...,
        [110.],
        [110.],
        [111.]],

       [[ 57.],
        [ 62.],
        [ 67.],
        ...,
        [110.],
        [111.],
        [110.]],

       [[ 57.],
        [ 61.],
        [ 65.],
        ...,
        [109.],
        [110.],
        [110.]],

       ...,

       [[ 72.],
        [ 72.],
        [ 73.],
        ...,
        [ 70.],
        [ 75.],
        [ 79.]],

       [[ 71.],
        [ 71.],
        [ 71.],
        ...,
        [ 82.],
        [ 85.],
        [ 87.]],

       [[ 72.],
        [ 71.],
        [ 70.],
        ...,
        [ 95.],
        [ 95.],
        [ 93.]]])>

In [17]:
model.summary()

Model: "sequential_112"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_224 (Conv2D)          (None, 45, 45, 128)       2176      
_________________________________________________________________
max_pooling2d_224 (MaxPoolin (None, 15, 15, 128)       0         
_________________________________________________________________
conv2d_225 (Conv2D)          (None, 12, 12, 128)       262272    
_________________________________________________________________
max_pooling2d_225 (MaxPoolin (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_112 (Flatten)        (None, 2048)              0         
_________________________________________________________________
dense_1344 (Dense)           (None, 512)               1049088   
_________________________________________________________________
dense_1345 (Dense)           (None, 128)            

In [16]:
model.fit(x=X_train_4000_tensor,y=y_train_4000,epochs=5, callbacks = [es]aa, verbose = 0)

TypeError: ignored

In [8]:
len(params_4000)

337

In [10]:
len(acc_4000)

336

In [11]:
params_csv = pd.DataFrame(params_4000)
params_csv.columns = ['b1', 'b2', 'lr', 'ck', 'mk', 'nf1', 'nf2', 'l']
acc_csv = pd.DataFrame(acc_4000)
acc_csv.columns = ['acc']

In [12]:
params_csv.to_csv('params.csv')
acc_csv.to_csv('acc.csv')

In [13]:
print(max(acc_4000))
print(params_4000[acc_4000.index(max(acc_4000))])

0.9621621370315552
[0.9, 0.999, 0.001, 0, 0, 64, 64, 0]


In [14]:
print(min(acc_4000))
print(params_4000[acc_4000.index(min(acc_4000))])

0.021052632480859756
[0.9, 0.0999, 0.001, 0, 0, 64, 128, 0]
